Import Statements

In [17]:
import numpy as np
import json
import csv
from sklearn.feature_extraction import text

Naive Bayes Class

In [103]:
class NaiveBayes(object):
    
    def __init__(self):
        # Load test file
        csv_file = open('KickStarterData.csv', encoding="utf-8")
        csv_reader = csv.reader(csv_file, delimiter=',')
        
        data_arr = np.empty((10, 15), dtype=object)
        counter = 0
        for row in csv_reader:
            if (counter < 10):
                temp_arr = np.array(row)
                data_arr[counter] = temp_arr
                counter += 1


In [104]:
nb = NaiveBayes()

<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
